In [32]:
import copy
import numpy as np
import sympy
from sympy import symbols, Matrix
sympy.init_printing()
%matplotlib inline
np.set_printoptions(precision=3, suppress=True, linewidth=250)

In [51]:
# 1-ELEMENT BEAM CLAMPED AT THE LEFT END. Everything is in a local system.

E = 2.1e5  # MPa
h = 10  # mm
b = 3  # mm
A = h * b
I = (h ** 3) * b / 12
EA = E * A
EI = E * I
L = 200  # mm

def element_stiffness(EA, EI, L):
    return np.matrix([
            np.array([EA / L, 0, 0, -EA / L, 0, 0]), 
            np.array([0, 12 * EI / (L ** 3), 6 * EI / (L ** 2), 0, -12 * EI / (L ** 3), 6 * EI / (L ** 2)]),
            np.array([0, 6 * EI / (L ** 2), 4 * EI / L, 0, -6 * EI / (L ** 2), 2 * EI / L]),
            np.array([-EA / L, 0, 0, EA / L, 0, 0]),
            np.array([0, -12 * EI / (L ** 3), -6 * EI / (L ** 2), 0, 12 * EI / (L ** 3), -6 * EI / (L ** 2)]),
            np.array([0, 6 * EI / (L ** 2), 2 * EI / L, 0, -6 * EI / (L ** 2), 4 * EI / L]),
        ])
ke = element_stiffness(EA, EI, L)
ke

matrix([[   31500.  ,        0.  ,        0.  ,   -31500.  ,        0.  ,        0.  ],
        [       0.  ,       78.75,     7875.  ,        0.  ,      -78.75,     7875.  ],
        [       0.  ,     7875.  ,  1050000.  ,        0.  ,    -7875.  ,   525000.  ],
        [  -31500.  ,        0.  ,        0.  ,    31500.  ,        0.  ,        0.  ],
        [       0.  ,      -78.75,    -7875.  ,        0.  ,       78.75,    -7875.  ],
        [       0.  ,     7875.  ,   525000.  ,        0.  ,    -7875.  ,  1050000.  ]])

In [34]:
# Load: 1 kN at the free end
g = 1000
q = np.matrix([[0, 0, 0, 0, -g, 0]]).T
# the load vector
q

matrix([[    0],
        [    0],
        [    0],
        [    0],
        [-1000],
        [    0]])

In [35]:
# adding BCs to node i
def add_BCs(_K):
    _ret = copy.deepcopy(_K)
    _ret[:3, :3] += np.matrix([[1e20, 0, 0], [0, 1e20, 0], [0, 0, 1e20]])
    return _ret
ke_console = add_BCs(_K=ke)
# the K matrix
ke_console

matrix([[  1.000e+20,   0.000e+00,   0.000e+00,  -3.150e+04,   0.000e+00,   0.000e+00],
        [  0.000e+00,   1.000e+20,   7.875e+03,   0.000e+00,  -7.875e+01,   7.875e+03],
        [  0.000e+00,   7.875e+03,   1.000e+20,   0.000e+00,  -7.875e+03,   5.250e+05],
        [ -3.150e+04,   0.000e+00,   0.000e+00,   3.150e+04,   0.000e+00,   0.000e+00],
        [  0.000e+00,  -7.875e+01,  -7.875e+03,   0.000e+00,   7.875e+01,  -7.875e+03],
        [  0.000e+00,   7.875e+03,   5.250e+05,   0.000e+00,  -7.875e+03,   1.050e+06]])

In [36]:
# solving the equation to get the nodal displacements
def solver(_K, _q):
    _ret = np.linalg.inv(_K) * _q
    return _ret
v = solver(_K=ke_console, _q=q)
v

matrix([[  0.   ],
        [ -0.   ],
        [ -0.   ],
        [  0.   ],
        [-50.794],
        [ -0.381]])

In [37]:
# nodal reaction forces
def nodal_reactions(_K, _v):
    return _K * _v
nr = nodal_reactions(_K=ke, _v=v)
nr

matrix([[      0.],
        [   1000.],
        [ 200000.],
        [      0.],
        [  -1000.],
        [     -0.]])

In [38]:
# reaction forces
def reactions(_nr, _q):
    return _nr - _q
r = reactions(_nr=nr, _q=q)
r

matrix([[      0.],
        [   1000.],
        [ 200000.],
        [      0.],
        [      0.],
        [     -0.]])